In [ ]:
import pandas as pd
import numpy as np

# reads in the data
df2 = pd.read_csv('/Users/eoinmurnaghan/Desktop/video_post_processing/log_data_cleaning/output.txt', delimiter="\t", header=None)

df2['Box_Number'], df2['Coords'] = df2[0].str.split(' ', 1).str

df2['Box_Number'] = df2[0].shift(-1)

# tentatively drops the Coords column
df2 = df2.drop('Coords', axis=1)

# renames the column headers
df2.columns = ['Test_Frame', 'Box_Number']

# splits the Box_Number column by single spacing
df2['Box_Number'], df2['Coords'] = df2['Box_Number'].str.split(' ', 1).str

# creates 4 new columns by splitting the Coords column by the comma
df2[['ymin','xmin', 'ymax', 'xmax']] = df2['Coords'].str.rsplit(',', expand=True)

df2.head()

In [ ]:
#add in functionality to remove box data from frame columns
magic = df2[ df2.Test_Frame.str.contains(",") & (df2['Box_Number'].notnull()) & df2['Coords'].isnull()].index
df2.drop(magic , inplace=True)

df2.head()

In [ ]:
#remove array seperator from ymin column
df2['ymin'] = df2['ymin'].astype(str).str.replace(r"[", '')

# remove array seperator from xmax column
df2['xmax'] = df2['xmax'].astype(str).str.replace(r"]", '')

# firmly drops the Coords column
df2 = df2.drop('Coords', axis=1)

# irons out the nan issue
df2.loc[df2.ymin == 'nan', 'ymin'] = np.nan
df2.loc[df2.xmax == 'nan', 'xmax'] = np.nan

# if the ymin value is NaN, change the Box_Number to NaN
df2.loc[df2.ymin == 'NaN', 'Box_Number'] = np.nan

df2.head()

In [ ]:
df2.dtypes

In [ ]:
#add in new columns where boolean outlines if box data lies in frame column
df2['helper'] = np.where(df2['Test_Frame'].str.contains(","), 'true', 'false')

# will be our final Frame column
df2['Frame'] = df2['Test_Frame']

#restart indexing
df2 = df2.reset_index(drop=True)

df2.head()

In [ ]:
df2.dtypes

In [ ]:
# for loop which stops frame number increasing when boolean is false
for i in range(1,len(df2)):
     if df2.helper[i] == 'true':
#             and df.Frame[i-1] == 381
             df2.Frame[i] = df2.Frame[i-1]
#              df.Frame[i] = df.Test_Frame[i]
             
     else:
        df2.Frame[i] = df2.Frame[i]

In [ ]:
# retains the columns we want
df2 = df2[['Frame', 'Box_Number', 'ymin', 'xmin', 'ymax', 'xmax']]

df2.head(342)

In [ ]:
df2['Frame'] = pd.to_numeric(df2['Frame'])

df2['Box_Number'] = pd.to_numeric(df2['Box_Number'])
df2['ymin'] = pd.to_numeric(df2['ymin'])
df2['xmin'] = pd.to_numeric(df2['xmin'])
df2['ymax'] = pd.to_numeric(df2['ymax'])
df2['xmax'] = pd.to_numeric(df2['xmax'])

df2.dtypes

In [ ]:
df3 = df2.set_index(['Frame', df2.groupby('Frame').cumcount()])[['ymin',
                                                               'xmin',
                                                               'ymax',
                                                               'xmax']].unstack().reset_index().rename_axis(None)



# drops the multilevel index
df3.columns = df3.columns.droplevel()

df3.head(343)



In [ ]:
df3.rename(columns={'xmin':'ymin1','xmin':'ymin2','ymax':'ymin3', 'xmax':'ymin4'}, inplace=True)
df3

In [ ]:
# outputs the data as csv
df3.to_csv('output_acceleration_new.csv', index=False)

In [ ]:
# box_data['Frame'].value_counts()
df2['freq'] = df2.groupby('Frame')['Frame'].transform('count')

df2.loc[df2['freq'] == 3]